In [1]:
import pandas as pd
from src.merging_with_id import *

In [2]:
# load datasets
datasets = {}
years = range(2016, 2024)
for year in years:
    print(f"EIE {year} Loading...")
    if int(year) >= 2019:
        file_name = f'Odata{year}File.csv'
    else:
        file_name = f'OpenData{year}.csv'
    try:           
        dataset = pd.read_csv(f"../../data_loader/{year}/{file_name}", sep=";", encoding='utf-8', dtype = str)
    except:
        dataset = pd.read_csv(f"../../data_loader/{year}/{file_name}", sep=";", encoding='Windows 1251', dtype = str)
    datasets.update({year:dataset})
    print("success")

EIE 2016 Loading...
success
EIE 2017 Loading...
success
EIE 2018 Loading...
success
EIE 2019 Loading...
success
EIE 2020 Loading...
success
EIE 2021 Loading...
success
EIE 2022 Loading...
success
EIE 2023 Loading...
success


In [3]:
for year, dataset in datasets.items():
    #lowercase all columns and add year as attribute
    dataset.columns = [col.lower() for col in dataset.columns]
    dataset['year'] = year

In [4]:
for year, dataset in datasets.items():
    print([col for col in dataset.columns if 'pt' in col])

['ukrptname', 'ukrptregname', 'ukrptareaname', 'ukrpttername', 'histptname', 'histptregname', 'histptareaname', 'histpttername', 'mathptname', 'mathptregname', 'mathptareaname', 'mathpttername', 'physptname', 'physptregname', 'physptareaname', 'physpttername', 'chemptname', 'chemptregname', 'chemptareaname', 'chempttername', 'bioptname', 'bioptregname', 'bioptareaname', 'biopttername', 'geoptname', 'geoptregname', 'geoptareaname', 'geopttername', 'engptname', 'engptregname', 'engptareaname', 'engpttername', 'frptname', 'frptregname', 'frptareaname', 'frpttername', 'deuptname', 'deuptregname', 'deuptareaname', 'deupttername', 'sptest', 'spteststatus', 'spptname', 'spptregname', 'spptareaname', 'sppttername', 'rusptname', 'rusptregname', 'rusptareaname', 'ruspttername']
['ukrptname', 'ukrptregname', 'ukrptareaname', 'ukrpttername', 'histptname', 'histptregname', 'histptareaname', 'histpttername', 'mathptname', 'mathptregname', 'mathptareaname', 'mathpttername', 'physptname', 'physptregna

In [5]:
test_centers_df = pd.DataFrame(columns=['ptname', 'ptregname', 'ptareaname', 'pttername', 'year'])

In [6]:
for year, dataset in datasets.items():
    prefixes = [col[:-6] for col in dataset.columns if 'ptname' in col]
    for prefix in prefixes:
        dataset_help = dataset.loc[:, [prefix + 'ptname', prefix + 'ptregname', prefix + 'ptareaname', prefix + 'pttername', 'year']]
        dataset_help.rename(columns={prefix + 'ptname': 'ptname', prefix + 'ptregname': 'ptregname', prefix + 'ptareaname': 'ptareaname', prefix + 'pttername': 'pttername'}, inplace=True)
        test_centers_df = pd.concat([test_centers_df, dataset_help], ignore_index=True)
        # print(year, prefix, dataset_help.dropna(how = 'all').shape[0])
    
test_centers_df

,ptname,ptregname,ptareaname,pttername,year
0,Мелітопольська загальноосвітня школа І-ІІІ сту...,Запорізька область,м.Мелітополь,м.Мелітополь,2016
1,"Красилівський навчально-виховний комплекс ""Гім...",Хмельницька область,Красилівський район,м.Красилів,2016
2,Першотравенська загальноосвітня школа І-ІІІ ст...,Дніпропетровська область,м.Першотравенськ,м.Першотравенськ,2016
3,Чернівецька загальноосвітня школа І-ІІІ ступен...,Чернівецька область,м.Чернівці. Шевченківський район міста,Шевченківський район міста,2016
4,Первомайська загальноосвітня школа I-III ступе...,Миколаївська область,м.Первомайськ,м.Первомайськ,2016
...,...,...,...,...,...
22924687,NaN,NaN,NaN,NaN,2021
22924688,NaN,NaN,NaN,NaN,2021
22924689,NaN,NaN,NaN,NaN,2021
22924690,NaN,NaN,NaN,NaN,2021


In [7]:
test_centers_df.columns

Index(['ptname', 'ptregname', 'ptareaname', 'pttername', 'year'], dtype='object')

In [8]:
test_centers_df.drop_duplicates(inplace=True)
# When the paricipant graduated in the previous years 
# the corresponding information about school (everything with 'eo') is NaN.
# We drop these rows.
test_centers_df.dropna(subset=test_centers_df.columns.difference(['year']), 
                      how = 'all', inplace=True)
test_centers_df.reset_index(drop=True, inplace=True)
test_centers_df

,ptname,ptregname,ptareaname,pttername,year
0,Мелітопольська загальноосвітня школа І-ІІІ сту...,Запорізька область,м.Мелітополь,м.Мелітополь,2016
1,"Красилівський навчально-виховний комплекс ""Гім...",Хмельницька область,Красилівський район,м.Красилів,2016
2,Першотравенська загальноосвітня школа І-ІІІ ст...,Дніпропетровська область,м.Першотравенськ,м.Першотравенськ,2016
3,Чернівецька загальноосвітня школа І-ІІІ ступен...,Чернівецька область,м.Чернівці. Шевченківський район міста,Шевченківський район міста,2016
4,Первомайська загальноосвітня школа I-III ступе...,Миколаївська область,м.Первомайськ,м.Первомайськ,2016
...,...,...,...,...,...
8860,Одеська загальноосвітня школа № 23 I-III ступе...,Одеська область,м.Одеса. Суворовський район міста,Суворовський район міста,2021
8861,Великодолинський заклад загальної середньої ос...,Одеська область,Овідіопольський район,смт Великодолинське,2021
8862,"Комунальний заклад освіти ""Спеціалізована школ...",Дніпропетровська область,м.Дніпро. Амур-Нижньодніпровський район міста,Амур-Нижньодніпровський район міста,2021
8863,"Комунальний заклад освіти ""Середня загальноосв...",Дніпропетровська область,м.Дніпро. Шевченківський район міста,Шевченківський район міста,2021


In [9]:
locations_df = pd.read_csv("./final_tables/locations.csv",  encoding='utf-8', dtype=str)
locations_df

,regname,areaname,tername,KOATUU_2020,KATOTTG_2023
0,Запорізька область,Мелітопольський район,с.Терпіння,2323085101,UA23080270010078454
1,Хмельницька область,Красилівський район,м.Красилів,6822710100,UA68040210010032567
2,Дніпропетровська область,Петропавлівський район,с.Дмитрівка,1223881501,UA12140170040016918
3,Чернівецька область,м.Чернівці,Шевченківський район міста,7310100000,UA73060610010033137
4,Миколаївська область,Врадіївський район,с.Кумарі,4822383001,UA48080050190079797
...,...,...,...,...,...
29214,США,м.Детройт,м.Детройт,0026010000,OC26010000000000000
29215,Велика Британія,м.Абериствіт,м.Абериствіт,0008030000,OC08030000000000000
29216,США,м.Нью-Йорк,м.Нью-Йорк,0026020000,OC26020000000000000
29217,США,м.Х'юстон,м.Х'юстон,0026040000,OC26040000000000000


In [10]:
locations_df.rename(columns={'regname':'ptregname', 'areaname':'ptareaname', 'tername':'pttername'}, inplace=True)
locations_df.columns

Index(['ptregname', 'ptareaname', 'pttername', 'KOATUU_2020', 'KATOTTG_2023'], dtype='object')

In [11]:
# add location codifier and aggregate by years
test_centers_df_loc = test_centers_df.merge(locations_df[['ptregname', 'ptareaname', 'pttername', 'KOATUU_2020']], on=['ptregname', 'ptareaname', 'pttername'], how='left')

test_centers_df_loc

,ptname,ptregname,ptareaname,pttername,year,KOATUU_2020
0,Мелітопольська загальноосвітня школа І-ІІІ сту...,Запорізька область,м.Мелітополь,м.Мелітополь,2016,2310700000
1,"Красилівський навчально-виховний комплекс ""Гім...",Хмельницька область,Красилівський район,м.Красилів,2016,6822710100
2,Першотравенська загальноосвітня школа І-ІІІ ст...,Дніпропетровська область,м.Першотравенськ,м.Першотравенськ,2016,1212600000
3,Чернівецька загальноосвітня школа І-ІІІ ступен...,Чернівецька область,м.Чернівці. Шевченківський район міста,Шевченківський район міста,2016,7310100000
4,Первомайська загальноосвітня школа I-III ступе...,Миколаївська область,м.Первомайськ,м.Первомайськ,2016,4810400000
...,...,...,...,...,...,...
8860,Одеська загальноосвітня школа № 23 I-III ступе...,Одеська область,м.Одеса. Суворовський район міста,Суворовський район міста,2021,5110137600
8861,Великодолинський заклад загальної середньої ос...,Одеська область,Овідіопольський район,смт Великодолинське,2021,5123755300
8862,"Комунальний заклад освіти ""Спеціалізована школ...",Дніпропетровська область,м.Дніпро. Амур-Нижньодніпровський район міста,Амур-Нижньодніпровський район міста,2021,1210136300
8863,"Комунальний заклад освіти ""Середня загальноосв...",Дніпропетровська область,м.Дніпро. Шевченківський район міста,Шевченківський район міста,2021,1210136600


In [12]:
test_centers_df_loc = test_centers_df_loc.loc[:, ['ptname', 'year', 'KOATUU_2020']]
test_centers_df_loc

,ptname,year,KOATUU_2020
0,Мелітопольська загальноосвітня школа І-ІІІ сту...,2016,2310700000
1,"Красилівський навчально-виховний комплекс ""Гім...",2016,6822710100
2,Першотравенська загальноосвітня школа І-ІІІ ст...,2016,1212600000
3,Чернівецька загальноосвітня школа І-ІІІ ступен...,2016,7310100000
4,Первомайська загальноосвітня школа I-III ступе...,2016,4810400000
...,...,...,...
8860,Одеська загальноосвітня школа № 23 I-III ступе...,2021,5110137600
8861,Великодолинський заклад загальної середньої ос...,2021,5123755300
8862,"Комунальний заклад освіти ""Спеціалізована школ...",2021,1210136300
8863,"Комунальний заклад освіти ""Середня загальноосв...",2021,1210136600


In [13]:
test_centers_df_loc[test_centers_df_loc.KOATUU_2020.isna()]

,ptname,year,KOATUU_2020


In [14]:
# add codifier for region
test_centers_df_loc['KOATUU_2020_reg'] = test_centers_df_loc['KOATUU_2020'].str[:2]
test_centers_df_loc

,ptname,year,KOATUU_2020,KOATUU_2020_reg
0,Мелітопольська загальноосвітня школа І-ІІІ сту...,2016,2310700000,23
1,"Красилівський навчально-виховний комплекс ""Гім...",2016,6822710100,68
2,Першотравенська загальноосвітня школа І-ІІІ ст...,2016,1212600000,12
3,Чернівецька загальноосвітня школа І-ІІІ ступен...,2016,7310100000,73
4,Первомайська загальноосвітня школа I-III ступе...,2016,4810400000,48
...,...,...,...,...
8860,Одеська загальноосвітня школа № 23 I-III ступе...,2021,5110137600,51
8861,Великодолинський заклад загальної середньої ос...,2021,5123755300,51
8862,"Комунальний заклад освіти ""Спеціалізована школ...",2021,1210136300,12
8863,"Комунальний заклад освіти ""Середня загальноосв...",2021,1210136600,12


In [15]:
test_centers_df_loc['ptname_temp'] = test_centers_df_loc['ptname']
test_centers_df_loc.loc[:,'ptname_temp'] = test_centers_df_loc['ptname_temp'].str.replace(r"[\(\[\{].*?[\)\]\}]", '', regex=True)
test_centers_df_loc.loc[:,'ptname_temp'] = test_centers_df_loc['ptname_temp'].str.replace(r'[\s\r\n\t]+', '', regex=True)
test_centers_df_loc

,ptname,year,KOATUU_2020,KOATUU_2020_reg,ptname_temp
0,Мелітопольська загальноосвітня школа І-ІІІ сту...,2016,2310700000,23,МелітопольськазагальноосвітняшколаІ-ІІІступені...
1,"Красилівський навчально-виховний комплекс ""Гім...",2016,6822710100,68,"Красилівськийнавчально-виховнийкомплекс""Гімназ..."
2,Першотравенська загальноосвітня школа І-ІІІ ст...,2016,1212600000,12,ПершотравенськазагальноосвітняшколаІ-ІІІступен...
3,Чернівецька загальноосвітня школа І-ІІІ ступен...,2016,7310100000,73,ЧернівецьказагальноосвітняшколаІ-ІІІступенів№2...
4,Первомайська загальноосвітня школа I-III ступе...,2016,4810400000,48,ПервомайськазагальноосвітняшколаI-IIIступенів№...
...,...,...,...,...,...
8860,Одеська загальноосвітня школа № 23 I-III ступе...,2021,5110137600,51,Одеськазагальноосвітняшкола№23I-IIIступенівОде...
8861,Великодолинський заклад загальної середньої ос...,2021,5123755300,51,Великодолинськийзакладзагальноїсередньоїосвіти...
8862,"Комунальний заклад освіти ""Спеціалізована школ...",2021,1210136300,12,"Комунальнийзакладосвіти""Спеціалізованашкола№55..."
8863,"Комунальний заклад освіти ""Середня загальноосв...",2021,1210136600,12,"Комунальнийзакладосвіти""Середнязагальноосвітня..."


In [16]:
# check whether there are duplicates by name and region
test_centers_df_loc[test_centers_df_loc.duplicated(subset=['ptname_temp', 'year', 'KOATUU_2020_reg'], keep=False)]

,ptname,year,KOATUU_2020,KOATUU_2020_reg,ptname_temp
18,Уманський державний педагогічний університет і...,2016,7110800000,71,Уманськийдержавнийпедагогічнийуніверситетімені...
76,"Національний технічний університет ""Харківськи...",2016,6310136600,63,"Національнийтехнічнийуніверситет""Харківськийпо..."
94,Валківський ліцей імені Олександра Масельськог...,2016,6321210100,63,ВалківськийліцейіменіОлександраМасельськогоВал...
116,Хіміко-технологічний коледж імені Івана Кожеду...,2016,5911000000,59,Хіміко-технологічнийколеджіменіІванаКожедубаШо...
135,Київський національний торговельно-економічний...,2016,8036400000,80,Київськийнаціональнийторговельно-економічнийун...
...,...,...,...,...,...
8828,Богородчанський ліцей №2 Богородчанської селищ...,2021,2620455100,26,Богородчанськийліцей№2Богородчанськоїселищноїр...
8830,Путильський заклад загальної середньої освіти ...,2021,7323555100,73,ПутильськийзакладзагальноїсередньоїосвітиІ-ІІІ...
8831,"Шумський навчально-виховний комплекс ""загально...",2021,6125810100,61,"Шумськийнавчально-виховнийкомплекс""загальноосв..."
8832,Заклад загальної середньої освіти І-ІІІ ступен...,2021,5121084201,51,ЗакладзагальноїсередньоїосвітиІ-ІІІступенівНер...


In [17]:
test_centers_df_loc =test_centers_df_loc.drop_duplicates(subset=['ptname_temp', 'year', 'KOATUU_2020_reg'], keep='first') 

In [18]:
test_centers_df_aggr = test_centers_df_loc.groupby(['ptname', 'KOATUU_2020', 'KOATUU_2020_reg'])['year'].agg(lambda x: list(x.unique())).reset_index()
test_centers_df_aggr

,ptname,KOATUU_2020,KOATUU_2020_reg,year
0,"""Запорізький національний університет"" Міністе...",2310136300,23,[2017]
1,"""Запорізький національний університет"" Міністе...",2310136300,23,[2018]
2,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2016]
3,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2017]
4,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2018]
...,...,...,...,...
3353,Ясінянська загальноосвітня школа І-ІІІ ступені...,2123656200,21,"[2016, 2017]"
3354,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,[2021]
3355,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,"[2017, 2018, 2019, 2020]"
3356,"комунальний заклад освіти ""Середня загальноосв...",1210137800,12,[2021]


In [19]:
#check whether len of location codifier is right
test_centers_df_aggr[test_centers_df_aggr.KOATUU_2020.str.len() != 10]

,ptname,KOATUU_2020,KOATUU_2020_reg,year


In [20]:
create_id(test_centers_df_aggr, 'ptname', 'id')

,ptname,KOATUU_2020,KOATUU_2020_reg,year,id
0,"""Запорізький національний університет"" Міністе...",2310136300,23,[2017],запорзькийнацональнийунверситетмнстерстваосвти...
1,"""Запорізький національний університет"" Міністе...",2310136300,23,[2018],запорзькийнацональнийунверситетмнстерстваосвти...
2,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2016],навчальновиховнийкомплексовруцькагмназяменандр...
3,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2017],навчальновиховнийкомплексовруцькагмназяменандр...
4,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2018],навчальновиховнийкомплексовруцькагмназяменандр...
...,...,...,...,...,...
3353,Ясінянська загальноосвітня школа І-ІІІ ступені...,2123656200,21,"[2016, 2017]",яснянськазагальноосвтняшкола3ступ2рахвськазака...
3354,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,[2021],загальноосвтняшкола93ступмзмалодеськаобл
3355,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,"[2017, 2018, 2019, 2020]",загальноосвтняшкола93ступмзмалодеськаобл
3356,"комунальний заклад освіти ""Середня загальноосв...",1210137800,12,[2021],середнязагальноосвтняшкола62


# MON data

## Full name

In [21]:
#load data
schools_data = pd.read_excel('./school_data/Schools_02.08.2023 16-56.xlsx', dtype = str)
schools_data

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,Код ЄДРПОУ,Статус,Тип закладу,Форма власності,Код КОАТУУ,Регіон,...,Телефон,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,(0800)330810,NaN,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)5264358,(044)5262002,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,"(044)234-50-55, (044)235-60-63",(044)234-50-55,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)259-31-60,(044)525-30-31,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)2584242,NaN,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,"Яблунівський навчально-виховний комплекс ""Зага...",144723,так,NaN,33329891,ліквідовано,навчально-виховний комплекс (об'єднання),Комунальна,7423886502,Чернігівська область,...,(046)422-43-40,NaN,solovejzanna8@gmail.com,NaN,Директор Новік Ольга Іванівна,ні,так,ні,ні,1
17331,"Ягіднянська філія опорного закладу ""Количівськ...",145351,так,NaN,43195202,ліквідовано,школа,Комунальна,7425582803,Чернігівська область,...,(0462)687733,NaN,yagidneschool@ukr.net,NaN,Завідувач філією Курилець Ніна Михайлівна,Філія,так,ні,ні,NaN
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,(04653)27571,NaN,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,(046)5924644,NaN,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90


In [22]:
schools_data.columns

Index(['Повна назва', 'Код в ЄДЕБО', 'ОУО підтвердив дані', 'Скорочена назва',
       'Код ЄДРПОУ', 'Статус', 'Тип закладу', 'Форма власності', 'Код КОАТУУ',
       'Регіон', 'Населений пункт', 'Місцезнаходження (юридична адреса)',
       'Головний заклад',
       'Найменування органу, до сфери управління якого належить заклад освіти',
       'Телефон', 'Факс', 'Електронна пошта', 'Веб-сайт', 'Керівник',
       'Опорний / Філія', 'Сільський', 'Гірський', 'Інтернат',
       'Ліцензовані обсяги'],
      dtype='object')

In [23]:
# create new with id
schools_data_id = schools_data.copy()
create_id(schools_data_id, 'Повна назва', 'id')

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,Код ЄДРПОУ,Статус,Тип закладу,Форма власності,Код КОАТУУ,Регіон,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,NaN,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN,приватнийзакладосвтикмдш2мки
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)5262002,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN,гмназя236мкив
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)234-50-55,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN,гмназя87мкив
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)525-30-31,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN,гмназя150мкив
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,NaN,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN,гмназя179мкив
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,"Яблунівський навчально-виховний комплекс ""Зага...",144723,так,NaN,33329891,ліквідовано,навчально-виховний комплекс (об'єднання),Комунальна,7423886502,Чернігівська область,...,NaN,solovejzanna8@gmail.com,NaN,Директор Новік Ольга Іванівна,ні,так,ні,ні,1,яблунвськийнавчальновиховнийкомплексзагальноос...
17331,"Ягіднянська філія опорного закладу ""Количівськ...",145351,так,NaN,43195202,ліквідовано,школа,Комунальна,7425582803,Чернігівська область,...,NaN,yagidneschool@ukr.net,NaN,Завідувач філією Курилець Ніна Михайлівна,Філія,так,ні,ні,NaN,ягднянськафляопорногозакладуколичвськазагально...
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,NaN,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1,ядутинськазагальноосвтняшкола3ступборзнянськач...
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,NaN,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90,янжулвськафля2ступсеменвськагмназ2семенвськаче...


In [24]:
schools_data_id.rename(columns={'Код КОАТУУ':'KOATUU_2020', 'Код ЄДРПОУ':'EDRPOU'}, inplace=True)
schools_data_id.columns

Index(['Повна назва', 'Код в ЄДЕБО', 'ОУО підтвердив дані', 'Скорочена назва',
       'EDRPOU', 'Статус', 'Тип закладу', 'Форма власності', 'KOATUU_2020',
       'Регіон', 'Населений пункт', 'Місцезнаходження (юридична адреса)',
       'Головний заклад',
       'Найменування органу, до сфери управління якого належить заклад освіти',
       'Телефон', 'Факс', 'Електронна пошта', 'Веб-сайт', 'Керівник',
       'Опорний / Філія', 'Сільський', 'Гірський', 'Інтернат',
       'Ліцензовані обсяги', 'id'],
      dtype='object')

In [25]:
schools_data_id[schools_data_id.KOATUU_2020.str.len() != 10]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id


In [26]:
schools_data_id[(schools_data_id.EDRPOU.str.len() < 8)&(schools_data_id.EDRPOU.str.len().notna())]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id
3906,Брониківська загальноосвітня школа I-II ступен...,145546,так,NaN,6669916,працює,школа,Комунальна,1824080401,Житомирська область,...,NaN,bronykyzosh@ukr.net,http://bronuku.at.ua/,О.М. Алєксєєва,ні,так,ні,ні,1,брониквськазагальноосвтняшкола2ступжитомирськаобл
3940,Вепринський навчально-виховний комплекс І ступеню,146135,так,NaN,2206723,працює,навчально-виховний комплекс (об'єднання),Комунальна,1825081101,Житомирська область,...,NaN,veprinnvk@ukr.net,NaN,Г.С. Миненко,ні,так,ні,ні,1,вепринськийнавчальновиховнийкомплексступен
4077,Калинівська загальноосвітня школа I-II ступені...,146872,так,NaN,6669980,працює,школа,Комунальна,1824084802,Житомирська область,...,NaN,kalinivka@i.ua,http://kalin-shkola.at.ua/,І.Й. Жигун,ні,так,ні,ні,1,калинвськазагальноосвтняшкола2ступжитомирськаобл
4118,Міжнародний приватний навчально-виховний компл...,149486,так,Міжнародний приватний НВК імені Януша Корчака,4068919,працює,навчально-виховний комплекс (об'єднання),Комунальна,1822510100,Житомирська область,...,NaN,grigory.khmelnytsky@gmail.com,NaN,NaN,ні,ні,ні,ні,1,мжнароднийприватнийнавчальновиховнийкомплексме...
4354,Селецька загальноосвітня школа I-II ступенів Н...,148312,так,NaN,6670486,працює,школа,Комунальна,1823787401,Житомирська область,...,NaN,school_selets@i.ua,NaN,І.В. Ніконенко,ні,так,ні,ні,1,селецьказагальноосвтняшкола2ступжитомирськаобл
15978,Зюбриська загальноосвітня школа І ступеня Мона...,140731,так,NaN,2363037,працює,школа,Комунальна,7123481701,Черкаська область,...,NaN,Zybr.chkola@gmail.com,NaN,NaN,ні,так,ні,ні,1,зюбриськазагальноосвтняшколаступмонастирищенсь...


In [27]:
schools_data_id['EDRPOU']=schools_data_id['EDRPOU'].apply(lambda x: '0'+x if (isinstance(x, str) and 0<len(x)<8) else x)

In [28]:
schools_data_id = schools_data_id[schools_data_id.EDRPOU.str.len() == 8]

In [29]:
schools_data_id.loc[:,'KOATUU_2020_reg'] = schools_data_id.loc[:,'KOATUU_2020'].str[:2]
schools_data_id

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_40554/1999010892.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schools_data_id.loc[:,'KOATUU_2020_reg'] = schools_data_id.loc[:,'KOATUU_2020'].str[:2]


,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN,приватнийзакладосвтикмдш2мки,80
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN,гмназя236мкив,80
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN,гмназя87мкив,80
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN,гмназя150мкив,80
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN,гмназя179мкив,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,"Яблунівський навчально-виховний комплекс ""Зага...",144723,так,NaN,33329891,ліквідовано,навчально-виховний комплекс (об'єднання),Комунальна,7423886502,Чернігівська область,...,solovejzanna8@gmail.com,NaN,Директор Новік Ольга Іванівна,ні,так,ні,ні,1,яблунвськийнавчальновиховнийкомплексзагальноос...,74
17331,"Ягіднянська філія опорного закладу ""Количівськ...",145351,так,NaN,43195202,ліквідовано,школа,Комунальна,7425582803,Чернігівська область,...,yagidneschool@ukr.net,NaN,Завідувач філією Курилець Ніна Михайлівна,Філія,так,ні,ні,NaN,ягднянськафляопорногозакладуколичвськазагально...,74
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1,ядутинськазагальноосвтняшкола3ступборзнянськач...,74
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90,янжулвськафля2ступсеменвськагмназ2семенвськаче...,74


In [30]:
schools_data_id[schools_data_id.duplicated(subset=['id', 'KOATUU_2020_reg', 'EDRPOU'], keep=False)]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
1611,Велимченський ліцей Велимченської сільської ра...,176569,так,Велимченський ліцей,21747814,ліквідовано,ліцей,Комунальна,0724280401,Волинська область,...,NaN,NaN,Директор Павлович Тетяна Володимирівна,ні,ні,ні,ні,NaN,велимченськийлцейвелимченськаволинськаобл,07
1612,Велимченський ліцей Велимченської сільської ра...,143856,так,Велимченський ліцей,21747814,працює,ліцей,Комунальна,0724280401,Волинська область,...,velimche__shcola@ukr.net,http://velimcheshkola.at.ua/,Павлович Тетяна Володимирівна,ні,так,ні,ні,NaN,велимченськийлцейвелимченськаволинськаобл,07


In [31]:
schools_data_id = schools_data_id.drop_duplicates(subset=['id', 'KOATUU_2020_reg', 'EDRPOU'], keep='last')

In [32]:
schools_data_id[schools_data_id.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
91,"Приватний заклад освіти ""УОЛД СКУЛ КАМПУС А""",151623,так,"ПЗО ""УОЛД СКУЛ КАМПУС А""",42180923,працює,школа,Приватна,8036300000,КИЇВ,...,campus.a@worldschool.com.ua,NaN,Директор Бадаєва Інна Миколаївна,ні,ні,ні,ні,NaN,приватнийзакладосвтиуолдскулкампус,80
254,Технічний ліцей м. Києва,145237,так,Технічний ліцей м. Києва,22876023,працює,ліцей,Комунальна,8036600000,КИЇВ,...,dtl-licej@ukr.ua,http://tl.kyiv.ua/,Директор Андрієнко Анатолій Михайлович,ні,ні,ні,ні,NaN,технчнийлцеймкив,80
268,"Приватний заклад освіти ""УОЛД СКУЛ КАМПУС Б""",176647,так,"ПЗО ""УОЛД СКУЛ КАМПУС Б""",43594721,працює,школа,Приватна,8000000000,КИЇВ,...,NaN,NaN,Директор Давидова Наталія Леонідівна,ні,ні,ні,ні,NaN,приватнийзакладосвтиуолдскулкампус,80
584,Технічний ліцей Шевченківського району м. Києва,149078,так,Технічний ліцей,22881946,працює,ліцей,Комунальна,8039100000,КИЇВ,...,texlyceum@ukr.net,techlyceum.kiev.ua,Директор Ілюшина Олена Олександрівна,ні,ні,ні,ні,844,технчнийлцеймкив,80
4728,Данилівський заклад загальної середньої освіти...,176502,так,Данилівський НВК І-ІІІ ступенів,40760974,працює,навчально-виховний комплекс (об'єднання),Комунальна,2125382601,Закарпатська область,...,danylovo@ukr.net,NaN,Директор Гайович Тетяна Василівна,ні,так,ні,ні,171,данилвський3ступхустськазакарпатськаобл,21
4729,Данилівський заклад загальної середньої освіти...,148256,так,ДАНИЛІВСЬКИЙ ЗЗСО І-ІІІ СТУПЕНІВ,34440724,реорганізовано,навчально-виховний комплекс (об'єднання),Комунальна,2125382601,Закарпатська область,...,danylovo@ukr.net,NaN,Директор Гайович Тетяна Василівна,ні,так,ні,ні,NaN,данилвський3ступхустськазакарпатськаобл,21
6818,Гоголівський ліцей Великодимерської селищної р...,146689,так,Гоголівський ліцей,25299425,працює,ліцей,Комунальна,3221281601,Київська область,...,gogoliv.school@gmail.com,http://gogolivska-zosh.kiev.sch.in.ua/,Директор Дяченко Оксана Іванівна,Опорний заклад,так,ні,ні,NaN,гоголвськийлцейвеликодимерськакивськаобл,32
6819,Гоголівський ліцей Великодимерської селищної р...,147130,так,філія Гоголівського ліцею,25299342,працює,ліцей,Комунальна,3221281601,Київська область,...,gogoliv-school2@ukr.net,http://gogoliv-school2.kiev.sch.in.ua,Завідувач філією Мишкоріз Любов Миколаївна,Філія,так,ні,ні,NaN,гоголвськийлцейвеликодимерськакивськаобл,32
11534,"Комунальний заклад ""Розсошенська гімназія Щерб...",139999,так,Розсошенська гімназія,21071866,працює,гімназія,Комунальна,5324087705,Полтавська область,...,rzsgymn@gmail.com,http://rzsschool.org.ua/,Директор Яловець Євген Олександрович,ні,так,ні,ні,834,розсошенськагмназящербанвськаполтавськаобл,53
11535,"Комунальний заклад ""Розсошенська гімназія Щерб...",151527,так,Розсошенська гімназія,52852882,ліквідовано,гімназія,Комунальна,5324087705,Полтавська область,...,rzsgymn@gmail.com,http://rzsschool.ucoz.ua/,Директор Яловець Євген Олександрович,ні,так,ні,ні,NaN,розсошенськагмназящербанвськаполтавськаобл,53


In [33]:
schools_data_id = schools_data_id[~schools_data_id.EDRPOU.isin(['34440724', '52852882', '58826264', 
                                                                '33329415', '26271178', '27646632', 
                                                                '04393999', '24623237', '25649174',
                                                                '25649172'])]

In [34]:
schools_data_id.drop(schools_data_id[schools_data_id.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)].index.values.tolist(), inplace=True)

In [35]:
test_centers_mon= test_centers_df_aggr.merge(schools_data_id[['KOATUU_2020_reg', 'EDRPOU', 'id']], on=['id', 'KOATUU_2020_reg'], how='left')
test_centers_mon.head()

,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU
0,"""Запорізький національний університет"" Міністе...",2310136300,23,[2017],запорзькийнацональнийунверситетмнстерстваосвти...,NaN
1,"""Запорізький національний університет"" Міністе...",2310136300,23,[2018],запорзькийнацональнийунверситетмнстерстваосвти...,NaN
2,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2016],навчальновиховнийкомплексовруцькагмназяменандр...,NaN
3,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2017],навчальновиховнийкомплексовруцькагмназяменандр...,NaN
4,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2018],навчальновиховнийкомплексовруцькагмназяменандр...,NaN


In [36]:
test_centers_mon.loc[test_centers_mon['EDRPOU'].isna(), 'EDRPOU'] = ''

In [37]:
test_centers_mon[test_centers_mon.EDRPOU != '']

,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU
5,"""Навчально-виховний комплекс ""Школа І-ІІ ступе...",8039100000,80,"[2018, 2021]",навчальновиховнийкомплексшкола2ступлцей38менвм...,22881780
6,"""Теребовлянська загальноосвітня школа І-ІІІ ст...",6125010100,61,[2021],теребовлянськазагальноосвтняшкола3ступ2щознахо...,24618748
54,Ірпінський ліцей №3 Ірпінської міської ради Ки...,3210900000,32,"[2020, 2021]",рпнськийлцей3рпнськакивськаобл,25667797
59,Іршавський заклад загальної середньої освіти І...,2121910100,21,"[2019, 2020, 2021]",ршавський3ступ2ршавськазакарпатськаобл,22099980
60,Іршавський ліцей Іршавської міської ради Закар...,2121910100,21,"[2019, 2020, 2021]",ршавськийлцейршавськазакарпатськаобл,33186757
...,...,...,...,...,...,...
3344,Яворівська гімназія ім.Осипа Маковея Яворівськ...,4625810100,46,"[2019, 2020]",яворвськагмназяменосипамаковеяяворвськальввськ...,22398598
3345,Яворівська гімназія імені Осипа Маковея Яворів...,4625810100,46,[2021],яворвськагмназяменосипамаковеяяворвськальввськ...,22398598
3347,Яворівський заклад загальної середньої освіти ...,4625810100,46,[2021],яворвський3ступ3ментарасашевченкаяворвськальвв...,22398718
3352,Яремчанський ліцей №1 Яремчанської міської рад...,2611000000,26,[2021],яремчанськийлцей1яремчанськаванофранквськаобл,20556113


## Short name

In [38]:
schools_data_id_short = schools_data_id[schools_data_id['Скорочена назва'].notna()]

In [39]:
create_id(schools_data_id_short, 'Скорочена назва', 'id')

/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/Python Course/ZNO-Dataset/notebooks/tables_creation/src/merging_with_id.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[id_] = dataset[id_].apply(del_rayon)
/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/Python Course/ZNO-Dataset/notebooks/tables_creation/src/merging_with_id.py:192: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[id_] = dataset[id_].apply(lambda s: s if (len(s) and s[

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN,приватнийзакладосвтикмдш2мки,80
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN,гмназя236,80
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN,гмназя87,80
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN,гмназя150,80
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN,гмназя179мкив,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,"комунальний заклад ""Яблунівський ліцей"" Черніг...",139767,так,"КЗ ""Яблунівський ліцей""",05266240,працює,ліцей,Комунальна,7424189701,Чернігівська область,...,Yablunovka@ukr.net,NaN,Директор Шаповаленко Неоніла Олександрівна,Опорний заклад,так,ні,так,170,яблунвськийлце,74
17329,Яблунівська загальноосвітня школа I-III ступен...,144175,так,Яблунівська ЗОШ І-ІІІ ступенів,33208204,працює,школа,Комунальна,7424189701,Чернігівська область,...,yablunivkask@ukr.net,NaN,Директор Колесник Анатолій Васильович,ні,так,ні,ні,320,яблунвськазагальноосвтняшкола3ступен,74
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1,ядутинськазагальноосвтняшкола3сту,74
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90,янжулвськафля2сту,74


In [40]:
schools_data_id_short[schools_data_id_short.duplicated(subset=['id', 'KOATUU_2020_reg', 'EDRPOU'], keep=False)]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg


In [41]:
schools_data_id_short =schools_data_id_short.drop_duplicates(subset=['id', 'EDRPOU', 'KOATUU_2020_reg'], keep='first') 

In [42]:
schools_data_id_short[schools_data_id_short.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
28,Спеціалізована школа І-ІІІ ступенів з поглибле...,148398,так,спеціалізована школа,22877005,працює,спеціалізована школа,Комунальна,8036100000,КИЇВ,...,sch15@i.ua,school15.kyiv.ua,Директор Литвин Валерій Олексійович,ні,ні,ні,ні,NaN,спецалзованашкол,80
496,ліцей №142 м. Києва,147801,так,ліцей,21622214,працює,ліцей,Комунальна,8038900000,КИЇВ,...,l142@ukr.net,NaN,Директор Сільвестрова Ірина Анатоліївна,ні,ні,ні,ні,600,лце,80
547,юридичний ліцей імені Ярослава Кондратьєва Нац...,147605,так,ліцей,35008569,працює,ліцей-інтернат,Державна,8038900000,КИЇВ,...,licei@naiau.kiev.ua,licey.naiau.kiev.ua,Ректор Чернєй Володимир Васильович,ні,ні,ні,ні,200,лце,80
572,Спеціалізована школа І-ІІІ ступенів №28 з погл...,149079,так,спеціалізована школа,22880792,працює,спеціалізована школа,Комунальна,8039100000,КИЇВ,...,school28kiev@ukr.net,sites.google.com/view/sch28,Директор Охріменко Наталія Миколаївна,ні,ні,ні,ні,1176,спецалзованашкол,80
652,Обласний науковий ліцей-інтернат Комунального ...,143380,так,Обласний науковий ліцей-інтернат,36364955,працює,науковий ліцей-інтернат,Комунальна,0510100000,Вінницька область,...,vinlitsey@ukr.net,https://www.vgpk.com.ua,Директор Войцехівський Костянтин Францович,ні,ні,ні,так,720,обласнийнауковийлцейнтерна,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15586,Рудська гімназія Смотрицької селищної ради Кам...,141431,так,Рудська гімназія,22777640,працює,гімназія,Комунальна,6821887701,Хмельницька область,...,rud.zosh2015@ukr.net,http://www.rudka-school.at.ua,Директор Середа Валентина Олексіївна,ні,так,ні,ні,NaN,рудськагмназ,68
16194,Рижавська філія Ладижинського ліцею Ладижинськ...,134934,так,Рижавська філія,24352264,працює,школа,Комунальна,7124387301,Черкаська область,...,rugavkaschool@gmail.com,NaN,Завідувач філією Квашук Оксана Миколаївна,Філія,ні,ні,ні,1,рижавськафл,71
16195,Рижавська філія Ладижинської загальноосвітньої...,151388,так,Рижавська філія,42469234,працює,школа,Комунальна,7124387301,Черкаська область,...,rugavkaschool@gmail.com,NaN,Завідувач філією Квашук Оксана Миколаївна,ні,так,ні,ні,1,рижавськафл,71
16551,Кам'янський ліцей Кам'янської сільської ради Ч...,142265,так,Кам'янський ліцей,21440097,працює,ліцей,Комунальна,7324584001,Чернівецька область,...,Kamjana-shkola@meta.ua,NaN,Директор Кожелянко Світлана Михайлівна,ні,так,ні,ні,550,камянськийлце,73


In [43]:
schools_data_id_short.drop(schools_data_id_short[schools_data_id_short.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)].index.values.tolist(), inplace=True)

In [44]:
test_centers_mon_final= merging(test_centers_mon, schools_data_id_short[['KOATUU_2020_reg', 'EDRPOU', 'id']], ['id', 'KOATUU_2020_reg'])
test_centers_mon_final

Percentage matched: 27.5 %


,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU
0,"""Запорізький національний університет"" Міністе...",2310136300,23,[2017],запорзькийнацональнийунверситетмнстерстваосвти...,
1,"""Запорізький національний університет"" Міністе...",2310136300,23,[2018],запорзькийнацональнийунверситетмнстерстваосвти...,
2,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2016],навчальновиховнийкомплексовруцькагмназяменандр...,
3,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2017],навчальновиховнийкомплексовруцькагмназяменандр...,
4,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2018],навчальновиховнийкомплексовруцькагмназяменандр...,
...,...,...,...,...,...,...
3353,Ясінянська загальноосвітня школа І-ІІІ ступені...,2123656200,21,"[2016, 2017]",яснянськазагальноосвтняшкола3ступ2рахвськазака...,
3354,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,[2021],загальноосвтняшкола93ступмзмалодеськаобл,
3355,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,"[2017, 2018, 2019, 2020]",загальноосвтняшкола93ступмзмалодеськаобл,
3356,"комунальний заклад освіти ""Середня загальноосв...",1210137800,12,[2021],середнязагальноосвтняшкола62,


# Universities

In [45]:
univ_data = pd.read_excel('./school_data/zakladi-vishchoyi-osviti-09.xlsx', dtype = str)
univ_data

,Назва закладу освіти,Код,Код головного закладу,Коротка назва,Назва закладу освіти (англ.),Рік заснування,Ідентифікаційний код,Вид закладу освіти,Тип закладу освіти,Форма власності,...,Поштовий індекс (юридична),Код КОАТУУ (юридична),Регіон (юридична),Населений пункт (юридична),Адреса (юридична),Телефон / факс,Електронна пошта,Веб-сайт,Посада керівника,"Прізвище, ім'я, по батькові керівника"
0,"Академія праці, соціальних відносин і туризму",53,NaN,АПСВТ,"Academy of Labour, Social Relations and Tourism",1993,04641405,Заклад вищої освіти,Академія,Приватна,...,03680,8000000000,КИЇВ,Київ,"вул. Велика Окружна дорога, 3",+38(044) 522-49-40,info@socosvita.kiev.ua,http://www. socosvita.kiev.ua,Ректор,Буяшенко Вікторія Василівна
1,Бізнес-коледж Приватного вищого навчального за...,3555,228,БК ПВНЗ «Європейський університет»,Business College of Private Higher Educational...,2002,40487422,Заклад вищої освіти,Коледж,Приватна,...,03115,8000000000,КИЇВ,Київ,"бульв. Академіка Вернадського, 16-В",+380(44)-432-00-58,bkeu@kyiv.e-u.in.ua,www.e-u.in.ua,директор,Сильницький Юрій Вікторович
2,"Вищий навчальний заклад ""Відкритий міжнародний...",217,NaN,"Університет ""Україна""",Higher Education Institution 'Open Internation...,1998,30373644,Заклад вищої освіти,Університет,Приватна,...,04071,8038500000,КИЇВ,Київ,"вул. Хорива, 1Г",(044) 4287410; 4252322; 4225019; 4252926,viddilorgrobotu@ukr.net,www.vmurol.com.ua,президент,Таланчук Петро Михайлович
3,"Вищий навчальний заклад Інститут дизайну, архі...",1319,NaN,"Вищий навчальний заклад ""Інститут дизайну, арх...",Higher educational institution “Institute of D...,2000,32594570,Заклад вищої освіти,Інститут,Приватна,...,03039,8038500000,КИЇВ,Київ,"вул. Фрометівська, 2",+380(44)-496-97-60,fb.mail.ir@gmail.com,www.ir.edu.ua,В. о. директора,Бойко Світлана Станіславівна
4,"Вищий навчальний заклад ""Київська Академія пер...",865,NaN,"ВНЗ ""КАПМ""","Higher educational institution ""Kyiv Academy o...",2002,26199252,Заклад вищої освіти,Академія,Приватна,...,03067,8038900000,КИЇВ,Київ,"вул. Виборзька, 86",0444555672,kapm@ukr.net,www.kapi.com.ua,ректор,Лубянська Світлана Павлівна
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,Відокремлений підрозділ національного універси...,2839,7,.,NaN,2000,35139007,Заклад вищої освіти,Відокремлений підрозділ,Державна,...,17400,7420610100,Чернігівська область,Бобровиця,"вул. Чернігівська, 19",NaN,NaN,NaN,NaN,NaN
1416,Борзнянський державний сільськогосподарський т...,390,NaN,БДСГТ,State agricultural technical school of Borzna,1999,00729178,Заклад вищої освіти,Технікум (училище),Державна,...,16400,7420810100,Чернігівська область,Борзна,"Олекси Десняка, 23",21106,koba_22@ukr.net,bdsgt.at.ua,Виконуючий обов`язки директора,Линник Віталій Володимирович
1417,Козелецький коледж ветеринарної медицини Білоц...,990,362,ККВМ БНАУ,"Kozelets College of Veterinary Medicine, Bila ...",1925,00729209,Заклад вищої освіти,Коледж,Державна,...,17000,7422055100,Чернігівська область,Козелець,"вул. Сім'ї Розумовських, 41","(046)462-12-00, (046)464-15-96, (046)462-14-83",techvet2@gmail.com,http://www.techvet.com.ua/,Директор коледжу,Редька Алла Іванівна
1418,Остерський коледж будівництва та дизайну,925,NaN,ОКБД,Oster college of civil engineering and designing,1945,01355708,Заклад вищої освіти,Коледж,Державна,...,17044,7422010400,Чернігівська область,Остер,"вул. 8 Березня, 7-А",(046) 464-32-85,oster_budtex@cg.ukrtel.net,http://okbd.16mb.com,Директор,Бардаченко Тетяна Миколаївна


In [46]:
univ_data_id = univ_data

In [47]:
create_id(univ_data_id, 'Назва закладу освіти', 'id')

,Назва закладу освіти,Код,Код головного закладу,Коротка назва,Назва закладу освіти (англ.),Рік заснування,Ідентифікаційний код,Вид закладу освіти,Тип закладу освіти,Форма власності,...,Код КОАТУУ (юридична),Регіон (юридична),Населений пункт (юридична),Адреса (юридична),Телефон / факс,Електронна пошта,Веб-сайт,Посада керівника,"Прізвище, ім'я, по батькові керівника",id
0,"Академія праці, соціальних відносин і туризму",53,NaN,АПСВТ,"Academy of Labour, Social Relations and Tourism",1993,04641405,Заклад вищої освіти,Академія,Приватна,...,8000000000,КИЇВ,Київ,"вул. Велика Окружна дорога, 3",+38(044) 522-49-40,info@socosvita.kiev.ua,http://www. socosvita.kiev.ua,Ректор,Буяшенко Вікторія Василівна,академяпрацсоцальнихвдносинтуризм
1,Бізнес-коледж Приватного вищого навчального за...,3555,228,БК ПВНЗ «Європейський університет»,Business College of Private Higher Educational...,2002,40487422,Заклад вищої освіти,Коледж,Приватна,...,8000000000,КИЇВ,Київ,"бульв. Академіка Вернадського, 16-В",+380(44)-432-00-58,bkeu@kyiv.e-u.in.ua,www.e-u.in.ua,директор,Сильницький Юрій Вікторович,бзнесколеджприватноговищогонавчальногозакладув...
2,"Вищий навчальний заклад ""Відкритий міжнародний...",217,NaN,"Університет ""Україна""",Higher Education Institution 'Open Internation...,1998,30373644,Заклад вищої освіти,Університет,Приватна,...,8038500000,КИЇВ,Київ,"вул. Хорива, 1Г",(044) 4287410; 4252322; 4225019; 4252926,viddilorgrobotu@ukr.net,www.vmurol.com.ua,президент,Таланчук Петро Михайлович,вищийнавчальнийзакладвдкритиймжнароднийунверси...
3,"Вищий навчальний заклад Інститут дизайну, архі...",1319,NaN,"Вищий навчальний заклад ""Інститут дизайну, арх...",Higher educational institution “Institute of D...,2000,32594570,Заклад вищої освіти,Інститут,Приватна,...,8038500000,КИЇВ,Київ,"вул. Фрометівська, 2",+380(44)-496-97-60,fb.mail.ir@gmail.com,www.ir.edu.ua,В. о. директора,Бойко Світлана Станіславівна,вищийнавчальнийзакладнститутдизайнуархтектурит...
4,"Вищий навчальний заклад ""Київська Академія пер...",865,NaN,"ВНЗ ""КАПМ""","Higher educational institution ""Kyiv Academy o...",2002,26199252,Заклад вищої освіти,Академія,Приватна,...,8038900000,КИЇВ,Київ,"вул. Виборзька, 86",0444555672,kapm@ukr.net,www.kapi.com.ua,ректор,Лубянська Світлана Павлівна,вищийнавчальнийзакладкивськаакадемяперукарсько...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,Відокремлений підрозділ національного універси...,2839,7,.,NaN,2000,35139007,Заклад вищої освіти,Відокремлений підрозділ,Державна,...,7420610100,Чернігівська область,Бобровиця,"вул. Чернігівська, 19",NaN,NaN,NaN,NaN,NaN,вдокремленийпдроздлнацональногоунверситетуборе...
1416,Борзнянський державний сільськогосподарський т...,390,NaN,БДСГТ,State agricultural technical school of Borzna,1999,00729178,Заклад вищої освіти,Технікум (училище),Державна,...,7420810100,Чернігівська область,Борзна,"Олекси Десняка, 23",21106,koba_22@ukr.net,bdsgt.at.ua,Виконуючий обов`язки директора,Линник Віталій Володимирович,борзнянськийдержавнийогосподарськийтехнку
1417,Козелецький коледж ветеринарної медицини Білоц...,990,362,ККВМ БНАУ,"Kozelets College of Veterinary Medicine, Bila ...",1925,00729209,Заклад вищої освіти,Коледж,Державна,...,7422055100,Чернігівська область,Козелець,"вул. Сім'ї Розумовських, 41","(046)462-12-00, (046)464-15-96, (046)462-14-83",techvet2@gmail.com,http://www.techvet.com.ua/,Директор коледжу,Редька Алла Іванівна,козелецькийколеджветеринарнамедициниблоцерквсь...
1418,Остерський коледж будівництва та дизайну,925,NaN,ОКБД,Oster college of civil engineering and designing,1945,01355708,Заклад вищої освіти,Коледж,Державна,...,7422010400,Чернігівська область,Остер,"вул. 8 Березня, 7-А",(046) 464-32-85,oster_budtex@cg.ukrtel.net,http://okbd.16mb.com,Директор,Бардаченко Тетяна Миколаївна,остерськийколеджбудвництватадизайн


In [48]:
univ_data_id.columns

Index(['Назва закладу освіти', 'Код', 'Код головного закладу', 'Коротка назва',
       'Назва закладу освіти (англ.)', 'Рік заснування',
       'Ідентифікаційний код', 'Вид закладу освіти', 'Тип закладу освіти',
       'Форма власності',
       'Найменування органу, до сфери управління якого належить заклад освіти',
       'Поштовий індекс (фактична)', 'Код КОАТУУ (фактична)',
       'Регіон (фактична)', 'Населений пункт (фактична)', 'Адреса (фактична)',
       'Поштовий індекс (юридична)', 'Код КОАТУУ (юридична)',
       'Регіон (юридична)', 'Населений пункт (юридична)', 'Адреса (юридична)',
       'Телефон / факс', 'Електронна пошта', 'Веб-сайт', 'Посада керівника',
       'Прізвище, ім'я, по батькові керівника', 'id'],
      dtype='object')

In [49]:
univ_data_id.rename(columns={'Код КОАТУУ (юридична)':'KOATUU_2020', 'Ідентифікаційний код':'EDRPOU'}, inplace=True)

In [50]:
univ_data_id = univ_data_id[univ_data_id.KOATUU_2020.str.len() == 10]

In [51]:
univ_data_id['KOATUU_2020_reg'] = univ_data_id['KOATUU_2020'].str[:2]
univ_data_id

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_40554/3478306073.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  univ_data_id['KOATUU_2020_reg'] = univ_data_id['KOATUU_2020'].str[:2]


,Назва закладу освіти,Код,Код головного закладу,Коротка назва,Назва закладу освіти (англ.),Рік заснування,EDRPOU,Вид закладу освіти,Тип закладу освіти,Форма власності,...,Регіон (юридична),Населений пункт (юридична),Адреса (юридична),Телефон / факс,Електронна пошта,Веб-сайт,Посада керівника,"Прізвище, ім'я, по батькові керівника",id,KOATUU_2020_reg
0,"Академія праці, соціальних відносин і туризму",53,NaN,АПСВТ,"Academy of Labour, Social Relations and Tourism",1993,04641405,Заклад вищої освіти,Академія,Приватна,...,КИЇВ,Київ,"вул. Велика Окружна дорога, 3",+38(044) 522-49-40,info@socosvita.kiev.ua,http://www. socosvita.kiev.ua,Ректор,Буяшенко Вікторія Василівна,академяпрацсоцальнихвдносинтуризм,80
1,Бізнес-коледж Приватного вищого навчального за...,3555,228,БК ПВНЗ «Європейський університет»,Business College of Private Higher Educational...,2002,40487422,Заклад вищої освіти,Коледж,Приватна,...,КИЇВ,Київ,"бульв. Академіка Вернадського, 16-В",+380(44)-432-00-58,bkeu@kyiv.e-u.in.ua,www.e-u.in.ua,директор,Сильницький Юрій Вікторович,бзнесколеджприватноговищогонавчальногозакладув...,80
2,"Вищий навчальний заклад ""Відкритий міжнародний...",217,NaN,"Університет ""Україна""",Higher Education Institution 'Open Internation...,1998,30373644,Заклад вищої освіти,Університет,Приватна,...,КИЇВ,Київ,"вул. Хорива, 1Г",(044) 4287410; 4252322; 4225019; 4252926,viddilorgrobotu@ukr.net,www.vmurol.com.ua,президент,Таланчук Петро Михайлович,вищийнавчальнийзакладвдкритиймжнароднийунверси...,80
3,"Вищий навчальний заклад Інститут дизайну, архі...",1319,NaN,"Вищий навчальний заклад ""Інститут дизайну, арх...",Higher educational institution “Institute of D...,2000,32594570,Заклад вищої освіти,Інститут,Приватна,...,КИЇВ,Київ,"вул. Фрометівська, 2",+380(44)-496-97-60,fb.mail.ir@gmail.com,www.ir.edu.ua,В. о. директора,Бойко Світлана Станіславівна,вищийнавчальнийзакладнститутдизайнуархтектурит...,80
4,"Вищий навчальний заклад ""Київська Академія пер...",865,NaN,"ВНЗ ""КАПМ""","Higher educational institution ""Kyiv Academy o...",2002,26199252,Заклад вищої освіти,Академія,Приватна,...,КИЇВ,Київ,"вул. Виборзька, 86",0444555672,kapm@ukr.net,www.kapi.com.ua,ректор,Лубянська Світлана Павлівна,вищийнавчальнийзакладкивськаакадемяперукарсько...,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,Відокремлений підрозділ національного універси...,2839,7,.,NaN,2000,35139007,Заклад вищої освіти,Відокремлений підрозділ,Державна,...,Чернігівська область,Бобровиця,"вул. Чернігівська, 19",NaN,NaN,NaN,NaN,NaN,вдокремленийпдроздлнацональногоунверситетуборе...,74
1416,Борзнянський державний сільськогосподарський т...,390,NaN,БДСГТ,State agricultural technical school of Borzna,1999,00729178,Заклад вищої освіти,Технікум (училище),Державна,...,Чернігівська область,Борзна,"Олекси Десняка, 23",21106,koba_22@ukr.net,bdsgt.at.ua,Виконуючий обов`язки директора,Линник Віталій Володимирович,борзнянськийдержавнийогосподарськийтехнку,74
1417,Козелецький коледж ветеринарної медицини Білоц...,990,362,ККВМ БНАУ,"Kozelets College of Veterinary Medicine, Bila ...",1925,00729209,Заклад вищої освіти,Коледж,Державна,...,Чернігівська область,Козелець,"вул. Сім'ї Розумовських, 41","(046)462-12-00, (046)464-15-96, (046)462-14-83",techvet2@gmail.com,http://www.techvet.com.ua/,Директор коледжу,Редька Алла Іванівна,козелецькийколеджветеринарнамедициниблоцерквсь...,74
1418,Остерський коледж будівництва та дизайну,925,NaN,ОКБД,Oster college of civil engineering and designing,1945,01355708,Заклад вищої освіти,Коледж,Державна,...,Чернігівська область,Остер,"вул. 8 Березня, 7-А",(046) 464-32-85,oster_budtex@cg.ukrtel.net,http://okbd.16mb.com,Директор,Бардаченко Тетяна Миколаївна,остерськийколеджбудвництватадизайн,74


In [52]:
univ_data_id[univ_data_id.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)]

,Назва закладу освіти,Код,Код головного закладу,Коротка назва,Назва закладу освіти (англ.),Рік заснування,EDRPOU,Вид закладу освіти,Тип закладу освіти,Форма власності,...,Регіон (юридична),Населений пункт (юридична),Адреса (юридична),Телефон / факс,Електронна пошта,Веб-сайт,Посада керівника,"Прізвище, ім'я, по батькові керівника",id,KOATUU_2020_reg


In [53]:
univ_data_id[(univ_data_id.EDRPOU.str.len() < 8)&(univ_data_id.EDRPOU.str.len().notna())]

,Назва закладу освіти,Код,Код головного закладу,Коротка назва,Назва закладу освіти (англ.),Рік заснування,EDRPOU,Вид закладу освіти,Тип закладу освіти,Форма власності,...,Регіон (юридична),Населений пункт (юридична),Адреса (юридична),Телефон / факс,Електронна пошта,Веб-сайт,Посада керівника,"Прізвище, ім'я, по батькові керівника",id,KOATUU_2020_reg


In [54]:
test_centers_univ = merging(test_centers_mon_final, univ_data_id[['KOATUU_2020_reg', 'id', 'EDRPOU']], ['KOATUU_2020_reg', 'id'])
test_centers_univ

Percentage matched: 38.4 %


,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU
0,"""Запорізький національний університет"" Міністе...",2310136300,23,[2017],запорзькийнацональнийунверситетмнстерстваосвти...,
1,"""Запорізький національний університет"" Міністе...",2310136300,23,[2018],запорзькийнацональнийунверситетмнстерстваосвти...,
2,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2016],навчальновиховнийкомплексовруцькагмназяменандр...,
3,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2017],навчальновиховнийкомплексовруцькагмназяменандр...,
4,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2018],навчальновиховнийкомплексовруцькагмназяменандр...,
...,...,...,...,...,...,...
3353,Ясінянська загальноосвітня школа І-ІІІ ступені...,2123656200,21,"[2016, 2017]",яснянськазагальноосвтняшкола3ступ2рахвськазака...,
3354,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,[2021],загальноосвтняшкола93ступмзмалодеськаобл,
3355,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,"[2017, 2018, 2019, 2020]",загальноосвтняшкола93ступмзмалодеськаобл,
3356,"комунальний заклад освіти ""Середня загальноосв...",1210137800,12,[2021],середнязагальноосвтняшкола62,


# Youcontrol

In [55]:
youcontrol =  pd.read_excel('./school_data/Youcontrol_historical_name.xlsx', dtype='str')
youcontrol 

,Код ЄДРПОУ,Назва,Коротка назва
0,00121146,"ВІДКРИТЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...",NaN
1,00121146,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ"""
2,00121146,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ"""
3,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",NaN
4,00131446,ІНШІ ОРГАНІЗАЦІЙНО-ПРАВОВІ ФОРМИ КОЛЕКТИВНЕ ПІ...,NaN
...,...,...,...
80921,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ"""
80923,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ"""
80924,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ"""


In [56]:
youcontrol.columns

Index(['Код ЄДРПОУ', 'Назва', 'Коротка назва'], dtype='object')

## long name

In [57]:
youcontrol[youcontrol.duplicated(subset=['Назва', 'Код ЄДРПОУ'], keep=False)]

,Код ЄДРПОУ,Назва,Коротка назва
3,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",NaN
5,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",КП МК-26
119,00452713,"ДЕРЖАВНИЙ НАВЧАЛЬНИЙ ЗАКЛАД ""ЗВЕНИГОРОДСЬКИЙ Ц...","ДНЗ ""ЗВЕНИГОРОДСЬКИЙ ЦППРК"""
121,00452713,"ДЕРЖАВНИЙ НАВЧАЛЬНИЙ ЗАКЛАД ""ЗВЕНИГОРОДСЬКИЙ Ц...","ДНЗ ""ЗВЕНИГОРОДСЬКИЙ ЦППРК"""
144,00854423,"КООПЕРАТИВ ""НОВОКРОПИВНИЦЬКИЙ""",NaN
...,...,...,...
80918,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО
80919,60043961,ДЕРЕБЧИНСЬКА СЕРЕДНЯ ЗАГАЛЬНООСВІТНЯ ШКОЛА I-I...,ДЕРЕБЧИНСЬКА СЗШ I-III СТУПЕНІВ
80921,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ"""


In [58]:
youcontrol_long = youcontrol.drop_duplicates(subset=['Назва', 'Код ЄДРПОУ'], keep='first')

In [59]:
create_id(youcontrol_long, 'Назва', 'id')

/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/Python Course/ZNO-Dataset/notebooks/tables_creation/src/merging_with_id.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:,id_] = dataset[attr]
/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/Python Course/ZNO-Dataset/notebooks/tables_creation/src/merging_with_id.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[id_] = dataset[id_].apply(del_rayon)
/Users/scipyguru/Library/Mob

,Код ЄДРПОУ,Назва,Коротка назва,id
0,00121146,"ВІДКРИТЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...",NaN,вдкритеакцонернетовариствопвдентеплоенергомонта
1,00121146,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",публчнеакцонернетовариствопвдентеплоенергомонта
2,00121146,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",приватнеакцонернетовариствопвдентеплоенергомонта
3,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",NaN,колективнепдпримствоблгородднстровськамеханзов...
4,00131446,ІНШІ ОРГАНІЗАЦІЙНО-ПРАВОВІ ФОРМИ КОЛЕКТИВНЕ ПІ...,NaN,колективнепдпримствоблгородднстровськамеханзов...
...,...,...,...,...
80918,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,деребчинськийопорний3ступвницькаобл
80920,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,деребчинськийопорний3ступвницькаобл
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинський3ступджуринськавниць...
80923,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинський3ступджуринськавниць...


In [60]:
youcontrol_long.rename(columns={'Код ЄДРПОУ':'EDRPOU'}, inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_40554/2371828141.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youcontrol_long.rename(columns={'Код ЄДРПОУ':'EDRPOU'}, inplace=True)


In [61]:
youcontrol_long[(youcontrol_long.EDRPOU.str.len() < 8)&(youcontrol_long.EDRPOU.str.len().notna())]

,EDRPOU,Назва,Коротка назва,id


In [62]:
youcontrol_long[youcontrol_long.duplicated(subset=['id', 'EDRPOU'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
3,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",NaN,колективнепдпримствоблгородднстровськамеханзов...
4,00131446,ІНШІ ОРГАНІЗАЦІЙНО-ПРАВОВІ ФОРМИ КОЛЕКТИВНЕ ПІ...,NaN,колективнепдпримствоблгородднстровськамеханзов...
48,00192471,"КОЛЕКТИВНЕ СПЕЦІАЛІЗОВАНЕ ПІДПРИЄМСТВО ""УКРЕНЕ...",NaN,колективнеспецалзованепдпримствоукренергочормет2
49,00192471,ІНШІ ОРГАНІЗАЦІЙНО-ПРАВОВІ ФОРМИ КОЛЕКТИВНЕ СП...,NaN,колективнеспецалзованепдпримствоукренергочормет2
67,00290280,"ВІДКРИТЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""НОВООЛЕКСАНДРІ...",NaN,вдкритеакцонернетовариствоновоолександрвськийц...
...,...,...,...,...
80917,60043961,ДЕРЕБЧИНСЬКА СЕРЕДНЯ ЗАГАЛЬНООСВІТНЯ ШКОЛА I-I...,ДЕРЕБЧИНСЬКА СЗШ I-III СТУПЕНІВ,деребчинськасереднязагальноосвтняшкола3ступвни...
80918,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,деребчинськийопорний3ступвницькаобл
80920,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,деребчинськийопорний3ступвницькаобл
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинський3ступджуринськавниць...


In [63]:
youcontrol_long = youcontrol_long.drop_duplicates(subset=['id', 'EDRPOU'], keep='first')

In [64]:
youcontrol_long[youcontrol_long.duplicated(subset=['id'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
221,01284979,МІЖШКІЛЬНИЙ НАВЧАЛЬНО-ВИРОБНИЧИЙ КОМБІНАТ,МНВК,мжшкльнийнавчальновиробничийкомбна
494,02218789,"КОМУНАЛЬНА ОРГАНІЗАЦІЯ (УСТАНОВА, ЗАКЛАД) МУЗИ...",NaN,музичнашкола2
498,02218884,"КОМУНАЛЬНА ОРГАНІЗАЦІЯ (УСТАНОВА, ЗАКЛАД) КОМУ...",NaN,музичнашкола3
503,02218921,"КОМУНАЛЬНА ОРГАНІЗАЦІЯ (УСТАНОВА, ЗАКЛАД) КОМУ...",NaN,музичнашкола2
719,02542129,МИКОЛАЇВСЬКИЙ ПРОФЕСІЙНИЙ ЛІЦЕЙ,NaN,миколавськийпрофесйнийлце
...,...,...,...,...
80889,45117367,"ФЕРМЕРСЬКЕ ГОСПОДАРСТВО ""ДОВІРА Б""","ФГ ""ДОВІРА Б""",фермерськегосподарстводовра
80903,45251511,Юріївська гімназія з початковою школою та дошк...,Юріївська гімназія,юрвськагмназязпочатковоюшколоютадошкльнимпдроз...
80905,45275530,Юріївська гімназія з початковою школою та дошк...,Юріївська гімназія,юрвськагмназязпочатковоюшколоютадошкльнимпдроз...
80907,45287518,Кобзарцівська початкова школа з дошкільним під...,Кобзарцівська початкова школа,кобзарцвськапочатковашколаздошкльнимпдроздломс...


In [65]:
youcontrol_long.drop(youcontrol_long[youcontrol_long.duplicated(subset=['id'], keep=False)].index.values.tolist()  , inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_40554/2842222147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youcontrol_long.drop(youcontrol_long[youcontrol_long.duplicated(subset=['id'], keep=False)].index.values.tolist()  , inplace=True)


In [66]:
test_centers_youcontrol_long = merging(test_centers_univ, youcontrol_long[['id', 'EDRPOU']], ['id'])
test_centers_youcontrol_long

Percentage matched: 80.3 %


,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU
0,"""Запорізький національний університет"" Міністе...",2310136300,23,[2017],запорзькийнацональнийунверситетмнстерстваосвти...,
1,"""Запорізький національний університет"" Міністе...",2310136300,23,[2018],запорзькийнацональнийунверситетмнстерстваосвти...,
2,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2016],навчальновиховнийкомплексовруцькагмназяменандр...,06670569
3,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2017],навчальновиховнийкомплексовруцькагмназяменандр...,06670569
4,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2018],навчальновиховнийкомплексовруцькагмназяменандр...,06670569
...,...,...,...,...,...,...
3353,Ясінянська загальноосвітня школа І-ІІІ ступені...,2123656200,21,"[2016, 2017]",яснянськазагальноосвтняшкола3ступ2рахвськазака...,
3354,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,[2021],загальноосвтняшкола93ступмзмалодеськаобл,22499879
3355,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,"[2017, 2018, 2019, 2020]",загальноосвтняшкола93ступмзмалодеськаобл,22499879
3356,"комунальний заклад освіти ""Середня загальноосв...",1210137800,12,[2021],середнязагальноосвтняшкола62,


## short name


In [67]:
youcontrol_short = youcontrol[youcontrol['Коротка назва'].notna()]

In [68]:
youcontrol_short.rename(columns={'Код ЄДРПОУ':'EDRPOU'}, inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_40554/692976443.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youcontrol_short.rename(columns={'Код ЄДРПОУ':'EDRPOU'}, inplace=True)


In [69]:
create_id(youcontrol_short, 'Коротка назва', 'id')

/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/Python Course/ZNO-Dataset/notebooks/tables_creation/src/merging_with_id.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:,id_] = dataset[attr]
/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/Python Course/ZNO-Dataset/notebooks/tables_creation/src/merging_with_id.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[id_] = dataset[id_].apply(del_rayon)
/Users/scipyguru/Library/Mob

,EDRPOU,Назва,Коротка назва,id
1,00121146,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",атпте
2,00121146,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",атпте
5,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",КП МК-26,кпмк26
7,00131564,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПРИКАРПАТТЯОБЛ...","ПАТ ""ПРИКАРПАТТЯОБЛЕНЕРГО""",патприкарпаттяобленерг
8,00131564,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПРИКАРПАТТЯОБЛ...","АТ ""ПРИКАРПАТТЯОБЛЕНЕРГО""",атприкарпаттяобленерг
...,...,...,...,...
80921,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,доззс
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинський3ступен
80923,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинський3ступен
80924,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинський3ступен


In [70]:
youcontrol_short[youcontrol_short.duplicated(subset=['Коротка назва', 'EDRPOU'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
1,00121146,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",атпте
2,00121146,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",атпте
11,00131954,"АКЦІОНЕРНА ТОВАРИСТВО ""ХАРКІВОБЛЕНЕРГО""","АТ ""ХАРКІВОБЛЕНЕРГО""",атхарквобленерг
12,00131954,"АКЦІОНЕРНЕ ТОВАРИСТВО ""ХАРКІВОБЛЕНЕРГО""","АТ ""ХАРКІВОБЛЕНЕРГО""",атхарквобленерг
62,00220434,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""НАУКОВО-ВИРОБН...","АТ ""НВАТ ""ВНДІКОМПРЕСОРМАШ""",атнватвндкомпресорма
...,...,...,...,...
80920,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,доззс
80921,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,доззс
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинський3ступен
80923,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинський3ступен


In [71]:
youcontrol_short = youcontrol_short.drop_duplicates(subset=['Коротка назва', 'EDRPOU'], keep='first')

In [72]:
youcontrol_short[youcontrol_short.duplicated(subset=['id', 'EDRPOU'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
103,00389357,ДЕРЖАВНИЙ НАВЧАЛЬНИЙ ЗАКЛАД ЛЬВІВСЬКЕ ВИЩЕ ПРО...,ДНЗ ЛЬВІВСЬКЕ ВПУХТ,днзльввськевпух
104,00389357,"ДЕРЖАВНИЙ НАВЧАЛЬНИЙ ЗАКЛАД ""ЛЬВІВСЬКЕ ВИЩЕ ПР...","ДНЗ ""ЛЬВІВСЬКЕ ВПУХТ""",днзльввськевпух
174,00951592,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ЄЛИЗАВЕТІВСЬК...","ПРАТ ""ЄЛИЗАВЕТІВСЬКИЙ ЕЛЕВАТОР""",пратлизаветвськийелевато
175,00951592,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ЄЛИЗАВЕТІВСЬКИ...","ПРАТ ""ЄЛИЗАВЕТІВСЬКИЙ ЕЛЕВАТОР""",пратлизаветвськийелевато
199,01204408,"""ПОЛТАВСЬКИЙ ОБЛАСНИЙ САНАТОРІЙ ДЛЯ ДІТЕЙ З ПО...","КП"" ПОДС""",кппод
...,...,...,...,...
80431,43861328,"НАЦІОНАЛЬНИЙ УНІВЕРСИТЕТ ""ОДЕСЬКА ПОЛІТЕХНІКА""",ОДЕСЬКА ПОЛІТЕХНІКА,одеськаполтехнк
80518,44129392,ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ПРИВАТ...,"ТОВ ПЗ ""ХЛ ""ХАНІ АКАДЕМІЯ"" ХО",пзхлханакадемях
80519,44129392,ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ПРИВАТ...,"ТОВ ПЗ ""ХЛ ""ХАНІ АКАДЕМІЯ"""" ХО",пзхлханакадемях
80581,44271671,РУБІЖАНСЬКИЙ ЛІЦЕЙ РУБІЖАНСЬКОЇ МІСЬКОЇ РАДИ Л...,РУБІЖАНСЬКИЙ ЛІЦЕЙ,рубжанськийлце


In [73]:
youcontrol_short = youcontrol_short.drop_duplicates(subset=['id', 'EDRPOU'], keep='first')

In [74]:
youcontrol_short[youcontrol_short.duplicated(subset=['id'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
221,01284979,МІЖШКІЛЬНИЙ НАВЧАЛЬНО-ВИРОБНИЧИЙ КОМБІНАТ,МНВК,мнв
295,01489345,ЛІЦЕЙ №18 ДЕСНЯНСЬКОГО РАЙОНУ МІСТА КИЄВА,ЛІЦЕЙ №18,лцей18
332,01730137,ОЛЕКСАНДРІВСЬКИЙ КОМБІНАТ КООПЕРАТИВНОЇ ПРОМИС...,ОЛЕКСАНДРІВСЬКИЙ ККП,олександрвськийкк
418,02137051,Ліцей с. Стрілки Стрілківської сільської ради ...,Стрілківський ліцей,стрлквськийлце
510,02218967,"КОМУНАЛЬНИЙ ЗАКЛАД ""МИСТЕЦЬКА ШКОЛА №2 КРАМАТО...",МИСТЕЦЬКА ШКОЛА № 2,мистецькашкола2
...,...,...,...,...
80905,45275530,Юріївська гімназія з початковою школою та дошк...,Юріївська гімназія,юрвськагмназ
80906,45277868,"Комунальний заклад ""Петрівська гімназія"" Кодим...","КЗ ""Петрівська гімназія""",петрвськагмназ
80907,45287518,Кобзарцівська початкова школа з дошкільним під...,Кобзарцівська початкова школа,кобзарцвськапочатковашкол
80912,45383386,Кобзарцівська початкова школа з дошкільним під...,Кобзарцівська початкова школа,кобзарцвськапочатковашкол


In [75]:
youcontrol_short.drop(youcontrol_short[youcontrol_short.duplicated(subset=['id'], keep=False)].index.values.tolist()  , inplace=True)

In [76]:
test_centers_youcontrol_short = merging(test_centers_youcontrol_long, youcontrol_short[['id', 'EDRPOU']], ['id'])
test_centers_youcontrol_short 

Percentage matched: 80.7 %


,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU
0,"""Запорізький національний університет"" Міністе...",2310136300,23,[2017],запорзькийнацональнийунверситетмнстерстваосвти...,
1,"""Запорізький національний університет"" Міністе...",2310136300,23,[2018],запорзькийнацональнийунверситетмнстерстваосвти...,
2,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2016],навчальновиховнийкомплексовруцькагмназяменандр...,06670569
3,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2017],навчальновиховнийкомплексовруцькагмназяменандр...,06670569
4,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2018],навчальновиховнийкомплексовруцькагмназяменандр...,06670569
...,...,...,...,...,...,...
3353,Ясінянська загальноосвітня школа І-ІІІ ступені...,2123656200,21,"[2016, 2017]",яснянськазагальноосвтняшкола3ступ2рахвськазака...,
3354,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,[2021],загальноосвтняшкола93ступмзмалодеськаобл,22499879
3355,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,"[2017, 2018, 2019, 2020]",загальноосвтняшкола93ступмзмалодеськаобл,22499879
3356,"комунальний заклад освіти ""Середня загальноосв...",1210137800,12,[2021],середнязагальноосвтняшкола62,22336611


# From schools

In [77]:
schools_data = pd.read_csv('./final_tables/schools_edrpou.csv',  dtype = str)
schools_data

,eoname,KOATUU_2020,year,EDRPOU
0,"Вiдокремлений структурний пiдроздiл ""Краматорс...",0510100000,[2023],04601943
1,"ВСП ""Технологічно-промисловий фаховий коледж В...",0510100000,[2022],00419667
2,Вище професійне училище №11 м. Вінниці,0510100000,"[2018, 2019, 2020, 2021, 2022, 2023]",03065891
3,Вище художнє професійно-технічне училище № 5 м...,0510100000,[2023],02539890
4,Вище художнє професійно-технічне училище №5 м....,0510100000,"[2018, 2019, 2020, 2021, 2022]",02539890
...,...,...,...,...
32055,Школа №25 І-ІІІ ступенів Шевченківського район...,8039100000,"[2016, 2017, 2018, 2019, 2020, 2021]",22880786
32056,Школа №27 І-ІІІ ступенів Шевченківського район...,8039100000,[2016],26125710
32057,Школа №70 І-ІІІ ступенів Шевченківського район...,8039100000,[2016],22881828
32058,Школа №95 І-ІІІ ступенів Шевченківського район...,8039100000,[2017],26125905


In [78]:
schools_data['KOATUU_2020_reg'] = schools_data['KOATUU_2020'].str[:2]

In [79]:
schools_data.columns

Index(['eoname', 'KOATUU_2020', 'year', 'EDRPOU', 'KOATUU_2020_reg'], dtype='object')

In [80]:
schools_data = schools_data[['eoname', 'KOATUU_2020', 'EDRPOU', 'KOATUU_2020_reg']]

In [81]:
schools_data['temp_eoname'] = schools_data['eoname'].str.replace(r'\r\n|\n', '', regex=True).str.strip()

In [82]:
schools_data[schools_data.duplicated(subset=['temp_eoname', 'EDRPOU', 'KOATUU_2020_reg'], keep=False)]

,eoname,KOATUU_2020,EDRPOU,KOATUU_2020_reg,temp_eoname
323,Войнашівська загальноосвітня школа I-III ступе...,0520280601,13326973,05,Войнашівська загальноосвітня школа I-III ступе...
326,Войнашівська загальноосвітня школа I-III ступе...,0520280603,13326973,05,Войнашівська загальноосвітня школа I-III ступе...
483,Степенівський навчально - виховний комплекс: з...,0520687603,23562485,05,Степенівський навчально - виховний комплекс: з...
484,Степенівський навчально - виховний комплекс: з...,0520687603,23562485,05,Степенівський навчально - виховний комплекс: з...
619,"Комунальний заклад ""Сербинівський навчально-ви...",0521085403,39977763,05,"Комунальний заклад ""Сербинівський навчально-ви..."
...,...,...,...,...,...
31717,"Приватний заклад передвищої освіти ""Мистецький...",8038500000,30728300,80,"Приватний заклад передвищої освіти ""Мистецький..."
31722,"Товариство з обмеженою відповідальністю ""Колег...",8038500000,33635570,80,"Товариство з обмеженою відповідальністю ""Колег..."
31831,Відокремлений структурний підрозділ Київський ...,8038900000,39867890,80,Відокремлений структурний підрозділ Київський ...
31939,Відокремлений структурний підрозділ Київський ...,8039100000,39867890,80,Відокремлений структурний підрозділ Київський ...


In [83]:
schools_data = schools_data.drop_duplicates(subset=['temp_eoname', 'EDRPOU', 'KOATUU_2020_reg'], keep='first')

In [84]:
schools_data[schools_data.duplicated(subset=['temp_eoname', 'KOATUU_2020_reg'], keep=False)]

,eoname,KOATUU_2020,EDRPOU,KOATUU_2020_reg,temp_eoname
18889,"Івашківський навчально-виховний комплекс ""Заг...",5122582601,33775772,51,"Івашківський навчально-виховний комплекс ""Зага..."
18890,"Івашківський навчально-виховний комплекс ""Зага...",5122582601,33775767,51,"Івашківський навчально-виховний комплекс ""Зага..."
18891,"Лабушненський навчально-виховний комплекс ""Заг...",5122583401,33775710,51,"Лабушненський навчально-виховний комплекс ""Заг..."
18892,"Лабушненський навчально-виховний комплекс ""Заг...",5122583401,26419446,51,"Лабушненський навчально-виховний комплекс ""Заг..."
19103,"Олександрівський навчально-виховний комплекс ""...",5123383707,34029703,51,"Олександрівський навчально-виховний комплекс ""..."
19104,"Олександрівський навчально-виховний комплекс ""...",5123383707,23863502,51,"Олександрівський навчально-виховний комплекс ""..."


In [85]:
schools_data = schools_data.drop_duplicates(subset=['temp_eoname', 'KOATUU_2020_reg'], keep='first')

In [86]:
test_centers_youcontrol_short['temp_eoname'] = test_centers_youcontrol_short['ptname'].str.replace(r'\r\n|\n', '', regex=True).str.strip()

In [87]:
test_centers_final = merging(test_centers_youcontrol_short, schools_data[['temp_eoname', 'EDRPOU', 'KOATUU_2020_reg']], ['temp_eoname', 'KOATUU_2020_reg'])
test_centers_final

Percentage matched: 95.9 %


,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU,temp_eoname
0,"""Запорізький національний університет"" Міністе...",2310136300,23,[2017],запорзькийнацональнийунверситетмнстерстваосвти...,,"""Запорізький національний університет"" Міністе..."
1,"""Запорізький національний університет"" Міністе...",2310136300,23,[2018],запорзькийнацональнийунверситетмнстерстваосвти...,,"""Запорізький національний університет"" Міністе..."
2,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2016],навчальновиховнийкомплексовруцькагмназяменандр...,06670569,"""Навчально-виховний комплекс ""Овруцька гімназі..."
3,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2017],навчальновиховнийкомплексовруцькагмназяменандр...,06670569,"""Навчально-виховний комплекс ""Овруцька гімназі..."
4,"""Навчально-виховний комплекс ""Овруцька гімназі...",1824210100,18,[2018],навчальновиховнийкомплексовруцькагмназяменандр...,06670569,"""Навчально-виховний комплекс ""Овруцька гімназі..."
...,...,...,...,...,...,...,...
3353,Ясінянська загальноосвітня школа І-ІІІ ступені...,2123656200,21,"[2016, 2017]",яснянськазагальноосвтняшкола3ступ2рахвськазака...,26326118,Ясінянська загальноосвітня школа І-ІІІ ступені...
3354,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,[2021],загальноосвтняшкола93ступмзмалодеськаобл,22499879,загальноосвітня школа №9 І – ІІІ ступенів міст...
3355,загальноосвітня школа №9 І – ІІІ ступенів міст...,5110600000,51,"[2017, 2018, 2019, 2020]",загальноосвтняшкола93ступмзмалодеськаобл,22499879,загальноосвітня школа №9 І – ІІІ ступенів міст...
3356,"комунальний заклад освіти ""Середня загальноосв...",1210137800,12,[2021],середнязагальноосвтняшкола62,22336611,"комунальний заклад освіти ""Середня загальноосв..."


In [88]:
test_centers_final[test_centers_final.EDRPOU == '']

,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU,temp_eoname
0,"""Запорізький національний університет"" Міністе...",2310136300,23,[2017],запорзькийнацональнийунверситетмнстерстваосвти...,,"""Запорізький національний університет"" Міністе..."
1,"""Запорізький національний університет"" Міністе...",2310136300,23,[2018],запорзькийнацональнийунверситетмнстерстваосвти...,,"""Запорізький національний університет"" Міністе..."
49,Інженерний інститут Запорізького національного...,2310136900,23,[2019],нженернийнститутзапорзькогонацональногоунверситет,,Інженерний інститут Запорізького національного...
50,Інститут післядипломної педагогічної освіти Ки...,8036600000,80,[2016],нститутпслядипломнапедагогчнаосвтикивськогоунв...,,Інститут післядипломної педагогічної освіти Ки...
65,Аграрний коледж управління і права Полтавської...,5310137000,53,"[2016, 2017]",аграрнийколеджуправлняправаполтавськадержавнаа...,,Аграрний коледж управління і права Полтавської...
...,...,...,...,...,...,...,...
3192,"Центральноукраїнський інститут ПрАТ ""ВНЗ"" ""МАУП""",3510136600,35,[2018],центральноукранськийнститутпратвнзмау,,"Центральноукраїнський інститут ПрАТ ""ВНЗ"" ""МАУП"""
3193,Центральноукраїнський інститут Приватне акціон...,3510136600,35,"[2019, 2020]",центральноукранськийнститутприватнеакцонернето...,,Центральноукраїнський інститут Приватне акціон...
3218,"Черкаський навчально-науковий інститут ДВНЗ ""У...",7110136400,71,"[2016, 2017, 2018]",черкаськийнавчальнонауковийнститутдвнзунверсит...,,"Черкаський навчально-науковий інститут ДВНЗ ""У..."
3252,Чернігівський національний педагогічний універ...,7410136300,74,"[2016, 2017, 2018]",чернгвськийнацональнийпедагогчнийунверситетмен...,,Чернігівський національний педагогічний універ...


In [89]:
test_centers_final.drop(columns=['temp_eoname'], inplace=True)

In [90]:
test_centers_final = test_centers_final.sort_values(by=['KOATUU_2020', 'ptname'])

In [91]:
# test_centers_final.to_csv('test_centers_edrpou.csv', index = False)

# by hand

In [92]:
data_hand = pd.read_csv('./school_data/testcenters_by_hand.csv',  dtype = str)
data_hand

,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU
0,Вище професійне училище №11 м. Вінниці,0510100000,05,"[2018, 2019, 2020, 2021]",вищепрофесйнеучилище11мвниц,03065891
1,"Відокремлений структурний підрозділ ""Вінницьки...",0510100000,05,[2021],вницькийколеджнацональногоунверситетухарчовихт...,35725833
2,Відокремлений структурний підрозділ Вінницький...,0510100000,05,[2021],вницькийколеджбудвництваархтектуритадизайнукив...,38814461
3,Вінницький коледж Національного університету х...,0510100000,05,"[2018, 2019, 2020]",вницькийколеджнацональногоунверситетухарчовихт...,35725833
4,Вінницький коледж будівництва і архітектури Ки...,0510100000,05,"[2016, 2017, 2018, 2019, 2020]",вницькийколеджбудвництваархтектурикивськогонац...,38814461
...,...,...,...,...,...,...
3476,Школа №199 І-ІІІ ступенів Шевченківського райо...,8039100000,80,[2016],школа1993ступмкив,22881923
3477,Школа №25 І-ІІІ ступенів Шевченківського район...,8039100000,80,"[2018, 2019, 2020, 2021]",школа253ступмкив,22880786
3478,Школа №70 І-ІІІ ступенів Шевченківського район...,8039100000,80,[2016],школа703ступмкив,22881828
3479,Школа №95 І-ІІІ ступенів Шевченківського район...,8039100000,80,[2017],школа953ступмкив,26125905


In [93]:
data_hand.columns

Index(['ptname', 'KOATUU_2020', 'KOATUU_2020_reg', 'year', 'id', 'EDRPOU'], dtype='object')

In [94]:
data_hand = data_hand[['ptname', 'KOATUU_2020', 'EDRPOU', 'KOATUU_2020_reg']]

In [95]:
data_hand['temp_eoname'] = data_hand['ptname'].str.replace(r'\r\n|\n', '', regex=True).str.strip()

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_40554/652156521.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_hand['temp_eoname'] = data_hand['ptname'].str.replace(r'\r\n|\n', '', regex=True).str.strip()


In [96]:
data_hand[data_hand.duplicated(subset=['temp_eoname', 'EDRPOU', 'KOATUU_2020_reg'], keep=False)]

,ptname,KOATUU_2020,EDRPOU,KOATUU_2020_reg,temp_eoname
162,"Луцький навчально-виховний комплекс ""Загальноо...",0710100000,26276366,07,"Луцький навчально-виховний комплекс ""Загальноо..."
163,"Луцький навчально-виховний комплекс ""Загальноо...",0710100000,26276366,07,"Луцький навчально-виховний комплекс ""Загальноо..."
196,Нововолинська загальноосвітня школа І-ІІІ ступ...,0710700000,20144391,07,Нововолинська загальноосвітня школа І-ІІІ ступ...
197,Нововолинська загальноосвітня школа І-ІІІ ступ...,0710700000,20144391,07,Нововолинська загальноосвітня школа І-ІІІ ступ...
272,"Вищий навчальний приватний заклад ""Дніпровськи...",1210136600,32495593,12,"Вищий навчальний приватний заклад ""Дніпровськи..."
...,...,...,...,...,...
2741,Чугуївська загальноосвітня школа I-III ступені...,6312000000,24335030,63,Чугуївська загальноосвітня школа I-III ступені...
2779,Херсонський політехнічний коледж Одеського нац...,6510136300,00237191,65,Херсонський політехнічний коледж Одеського нац...
2800,Херсонський політехнічний коледж Одеського нац...,6510136900,00237191,65,Херсонський політехнічний коледж Одеського нац...
2942,"Філія ""Черкаський кооперативний економіко-прав...",7110136400,39843664,71,"Філія ""Черкаський кооперативний економіко-прав..."


In [97]:
data_hand = data_hand.drop_duplicates(subset=['temp_eoname', 'EDRPOU', 'KOATUU_2020_reg'], keep='first')

In [98]:
data_hand[data_hand.duplicated(subset=['temp_eoname', 'KOATUU_2020_reg'], keep=False)]

,ptname,KOATUU_2020,EDRPOU,KOATUU_2020_reg,temp_eoname


In [99]:
test_centers_final['temp_eoname'] = test_centers_final['ptname'].str.replace(r'\r\n|\n', '', regex=True).str.strip()

In [100]:
test_centers = merging(test_centers_final, data_hand[['temp_eoname', 'EDRPOU', 'KOATUU_2020_reg']], ['temp_eoname', 'KOATUU_2020_reg'])
test_centers

Percentage matched: 100.0 %


,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU,temp_eoname
0,Вище професійне училище №11 м. Вінниці,0510100000,05,"[2018, 2019, 2020, 2021]",вищепрофесйнеучилище11мвниц,03065891,Вище професійне училище №11 м. Вінниці
1,"Відокремлений структурний підрозділ ""Вінницьки...",0510100000,05,[2021],вницькийколеджнацональногоунверситетухарчовихт...,35725833,"Відокремлений структурний підрозділ ""Вінницьки..."
2,Відокремлений структурний підрозділ Вінницький...,0510100000,05,[2021],вницькийколеджбудвництваархтектуритадизайнукив...,38814461,Відокремлений структурний підрозділ Вінницький...
3,Вінницький коледж Національного університету х...,0510100000,05,"[2018, 2019, 2020]",вницькийколеджнацональногоунверситетухарчовихт...,35725833,Вінницький коледж Національного університету х...
4,Вінницький коледж будівництва і архітектури Ки...,0510100000,05,"[2016, 2017, 2018, 2019, 2020]",вницькийколеджбудвництваархтектурикивськогонац...,38814461,Вінницький коледж будівництва і архітектури Ки...
...,...,...,...,...,...,...,...
3353,Школа №199 І-ІІІ ступенів Шевченківського райо...,8039100000,80,[2016],школа1993ступмкив,22881923,Школа №199 І-ІІІ ступенів Шевченківського райо...
3354,Школа №25 І-ІІІ ступенів Шевченківського район...,8039100000,80,"[2018, 2019, 2020, 2021]",школа253ступмкив,22880786,Школа №25 І-ІІІ ступенів Шевченківського район...
3355,Школа №70 І-ІІІ ступенів Шевченківського район...,8039100000,80,[2016],школа703ступмкив,22881828,Школа №70 І-ІІІ ступенів Шевченківського район...
3356,Школа №95 І-ІІІ ступенів Шевченківського район...,8039100000,80,[2017],школа953ступмкив,26125905,Школа №95 І-ІІІ ступенів Шевченківського район...


In [101]:
test_centers.drop(columns=['temp_eoname'], inplace=True)

In [102]:
test_centers = test_centers.sort_values(by=['KOATUU_2020', 'ptname'])

In [103]:
# test_centers.to_csv('testcenters_edrpou_not checked.csv', index = False)

# Check

In [104]:
test_centers_to_check = test_centers
test_centers_to_check['year_s'] = test_centers_to_check['year'].astype(str)

In [105]:
test_centers_to_check[test_centers_to_check.duplicated(subset=['year_s', 'EDRPOU', 'ptname'], keep=False)]

,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU,year_s


In [106]:
result_df = test_centers_to_check.groupby('EDRPOU')['year'].apply(lambda x: sorted(y for sublist in x for y in sublist)).reset_index()

result_df.columns

Index(['EDRPOU', 'year'], dtype='object')

In [107]:
duplicates_mask = result_df['year'].apply(lambda x: len(x) != len(set(x)))

# Filter rows with duplicates
duplicates_df = result_df[duplicates_mask]

duplicates_df 

,EDRPOU,year
13,00208752,"[2016, 2016, 2017, 2018, 2019, 2020, 2021]"
78,01597997,"[2016, 2017, 2018, 2019, 2020, 2021, 2021]"
130,02125243,"[2016, 2017, 2018, 2019, 2019, 2020, 2020, 202..."
274,05460798,"[2016, 2016, 2017, 2017, 2018, 2018, 2019, 202..."
277,05536076,"[2016, 2017, 2018, 2018, 2019, 2019, 2020, 202..."
359,14163438,"[2016, 2016, 2017, 2017, 2018, 2018, 2019, 201..."
506,20933314,"[2016, 2018, 2019, 2019, 2020, 2020, 2021, 2021]"
578,21431276,"[2018, 2019, 2020, 2020, 2021, 2021]"
1044,23210450,"[2016, 2017, 2018, 2020, 2021, 2021]"
1120,23863488,"[2016, 2017, 2021, 2021]"


In [108]:
duplicates_df.EDRPOU.unique()

array(['00208752', '01597997', '02125243', '05460798', '05536076',
       '14163438', '20933314', '21431276', '23210450', '23863488',
       '24952840', '25322733', '26363711', '26395881', '26508374',
       '33297907', '34227267', '37664469', '39568620', '43153403'],
      dtype=object)

In [109]:
problems_mask = test_centers_to_check['EDRPOU'].apply(lambda x: x in duplicates_df.EDRPOU.unique())

# Filter rows with duplicates
problems = test_centers_to_check[problems_mask]

problems

,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU,year_s
260,"Університет митної справи та фінансів ННІ ""Фін...",1210136300,12,"[2017, 2018, 2019, 2020]",унверситетмитнасправитафнансвнфнансоваакадем,39568620,"[2017, 2018, 2019, 2020]"
264,Державний професійно-технічний навчальний закл...,1210136600,12,"[2019, 2020]",державнийпрофесйнотехнчнийнавчальнийзакладднпр...,05536076,"[2019, 2020]"
265,Дніпропетровський регіональний центр професійн...,1210136600,12,"[2017, 2018]",днпропетровськийрегональнийцентрпрофесйнотехнч...,05536076,"[2017, 2018]"
308,Університет митної справи та фінансів (корпус 2),1210136900,12,[2021],унверситетмитнасправитафнанс,39568620,[2021]
309,"Університет митної справи та фінансів ННІ ""Мит...",1210136900,12,"[2016, 2021]",унверситетмитнасправитафнансвнмитнаакадем,39568620,"[2016, 2021]"
...,...,...,...,...,...,...,...
2935,Чернівецька загальноосвітня школа І-ІІІ ступен...,7310100000,73,"[2018, 2019, 2020, 2021]",чернвецьказагальноосвтняшкола3ступ28чернвецькао,21431276,"[2018, 2019, 2020, 2021]"
3000,Навчально-науковий інститут економіки Чернігів...,7410136300,74,"[2016, 2017, 2018]",навчальнонауковийнститутекономкичернгвськогона...,05460798,"[2016, 2017, 2018]"
3001,Навчально-науковий інститут технологій Чернігі...,7410136300,74,"[2016, 2017, 2018, 2019]",навчальнонауковийнституттехнологйчернгвськогон...,05460798,"[2016, 2017, 2018, 2019]"
3002,"Національний університет ""Чернігівська політех...",7410136300,74,"[2020, 2021]",нацональнийунверситетчернгвськаполтехнк,05460798,"[2020, 2021]"


In [110]:
# problems.to_csv('test_centers_problems.csv', index = False)

# Solve issues with wrong EDRPOU

In [111]:
issues = pd.read_csv('./school_data/test_centers_problems.csv',  dtype = str)
issues = issues[issues.to_change.notna()]
issues = issues[['ptname', 'KOATUU_2020', 'EDRPOU', 'to_change']]
issues

,ptname,KOATUU_2020,EDRPOU,to_change
7,"Відокремлений структурний підрозділ ""Херсонськ...",6510136900,01597997,00237191
29,Чернівецька гімназія №7 Чернівецької міської ради,7310100000,21431276,35529258
41,Школа І-ІІІ ступенів №29,8038000000,23887299,22877815
59,Середня загальноосвітня школа №98,4610136800,26508374,22335936


In [112]:
test_centers_fixed = test_centers
test_centers_fixed

,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU,year_s
0,Вище професійне училище №11 м. Вінниці,0510100000,05,"[2018, 2019, 2020, 2021]",вищепрофесйнеучилище11мвниц,03065891,"[2018, 2019, 2020, 2021]"
1,"Відокремлений структурний підрозділ ""Вінницьки...",0510100000,05,[2021],вницькийколеджнацональногоунверситетухарчовихт...,35725833,[2021]
2,Відокремлений структурний підрозділ Вінницький...,0510100000,05,[2021],вницькийколеджбудвництваархтектуритадизайнукив...,38814461,[2021]
3,Вінницький коледж Національного університету х...,0510100000,05,"[2018, 2019, 2020]",вницькийколеджнацональногоунверситетухарчовихт...,35725833,"[2018, 2019, 2020]"
4,Вінницький коледж будівництва і архітектури Ки...,0510100000,05,"[2016, 2017, 2018, 2019, 2020]",вницькийколеджбудвництваархтектурикивськогонац...,38814461,"[2016, 2017, 2018, 2019, 2020]"
...,...,...,...,...,...,...,...
3353,Школа №199 І-ІІІ ступенів Шевченківського райо...,8039100000,80,[2016],школа1993ступмкив,22881923,[2016]
3354,Школа №25 І-ІІІ ступенів Шевченківського район...,8039100000,80,"[2018, 2019, 2020, 2021]",школа253ступмкив,22880786,"[2018, 2019, 2020, 2021]"
3355,Школа №70 І-ІІІ ступенів Шевченківського район...,8039100000,80,[2016],школа703ступмкив,22881828,[2016]
3356,Школа №95 І-ІІІ ступенів Шевченківського район...,8039100000,80,[2017],школа953ступмкив,26125905,[2017]


In [113]:
merged_df = test_centers_fixed.merge(issues, on=['ptname', 'KOATUU_2020', 'EDRPOU'], how='left')
merged_df[merged_df['to_change'].notna()]

,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU,year_s,to_change
1548,Середня загальноосвітня школа №98,4610136800,46,[2016],середнязагальноосвтняшкола98,26508374,[2016],22335936
2695,"Відокремлений структурний підрозділ ""Херсонськ...",6510136900,65,[2021],херсонськийполтехнчнийколедждержавногоунверсит...,01597997,[2021],00237191
2929,Чернівецька гімназія №7 Чернівецької міської ради,7310100000,73,"[2020, 2021]",чернвецькагмназя7чернвецькао,21431276,"[2020, 2021]",35529258


In [114]:
merged_df.loc[merged_df['to_change'].notna(), 'EDRPOU'] = merged_df.loc[merged_df['to_change'].notna(), 'to_change']
merged_df[merged_df['to_change'].notna()]

,ptname,KOATUU_2020,KOATUU_2020_reg,year,id,EDRPOU,year_s,to_change
1548,Середня загальноосвітня школа №98,4610136800,46,[2016],середнязагальноосвтняшкола98,22335936,[2016],22335936
2695,"Відокремлений структурний підрозділ ""Херсонськ...",6510136900,65,[2021],херсонськийполтехнчнийколедждержавногоунверсит...,00237191,[2021],00237191
2929,Чернівецька гімназія №7 Чернівецької міської ради,7310100000,73,"[2020, 2021]",чернвецькагмназя7чернвецькао,35529258,"[2020, 2021]",35529258


In [115]:
merged_df.columns

Index(['ptname', 'KOATUU_2020', 'KOATUU_2020_reg', 'year', 'id', 'EDRPOU',
       'year_s', 'to_change'],
      dtype='object')

In [116]:
merged_df = merged_df[['ptname', 'KOATUU_2020', 'year', 'EDRPOU']]

In [117]:
merged_df.to_csv('./final_tables/test_centers_edrpou.csv', index = False)